<a href="https://colab.research.google.com/github/stsmith1991/Adv_DS_libs/blob/main/HW6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Домашнее задание к урокам 4-6**

In [119]:
#Импортирую библиoтеки
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score as r2
from matplotlib import pyplot as plt
%matplotlib inline
df = load_boston()
all = df["data"]
observations = df["feature_names"]
target = df["target"]
X = pd.DataFrame(all, columns=observations)
y = pd.DataFrame(target, columns=["price"])

Задание 1.

In [120]:
#Создаю обучающий фрем и массив с таргетом
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=42)
#Создаю экземпляр объекта линейной регресси, передаю ему обучающую выборку и осуществляю обучение
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
#Вывожу полученные интерсепты
y_pred[0, 0]

28.648960046324063

In [121]:
#Преобразую в массив и сравниваю точное с предсказанным
check= pd.DataFrame({
    "Фактическое": y_test["price"],
    "Предсказанное": y_pred.flatten()
})
check.head(10)

,Фактическое,Предсказанное
173,23.6,28.648960
274,32.4,36.495014
491,13.6,15.411193
72,22.8,25.403213
452,16.1,18.855280
76,20.0,23.146689
316,17.8,17.392124
140,14.0,14.078599
471,19.6,23.036927
500,16.8,20.599433


In [122]:
#Проверяю методом R-квадрат
ln_miss=r2(y_test, y_pred)
print(f'Поргрешность предсказаний линейной регрессии составляет {round(ln_miss, 2)} единиц')

Поргрешность предсказаний линейной регрессии составляет 0.71 единиц


Задание 2.

In [123]:
#Загружаю требуемые библиотеки
from sklearn.ensemble import RandomForestRegressor

In [124]:
#Инициация с требуемыми параметрами
model=RandomForestRegressor(n_estimators=1000, max_depth=12, random_state=42)
#Обучение модели
model.fit(X_train,  y_train.values[:, 0])
#Результаты и предварительная оценка
y_forest = model.predict(X_test)
second= pd.DataFrame({
    "Фактическое": y_test["price"],
    "Предсказанное": y_forest.flatten()
})
second.head(10)

,Фактическое,Предсказанное
173,23.6,22.846138
274,32.4,31.156114
491,13.6,16.297226
72,22.8,23.821036
452,16.1,17.212148
76,20.0,21.820092
316,17.8,19.866369
140,14.0,14.759938
471,19.6,21.235224
500,16.8,20.883103


In [132]:
#сравниваю погрешности предсказаний
forest_miss=r2(y_test, y_forest)
result=pd.merge(left=check, right=second, left_on='Фактическое', right_on='Фактическое', suffixes=(' Лин.рег', '" Лес"'))
result.head(3)

,Фактическое,Предсказанное Лин.рег,"Предсказанное"" Лес"""
0,23.6,28.648960,22.846138
1,32.4,36.495014,31.156114
2,13.6,15.411193,16.297226


In [126]:
if ln_miss>forest_miss:
  print('Предсказание методом случайного леса точнее')
elif ln_miss<forest_miss:
  print('Предсказание методом линейной регрессии точнее')
else:
  print('Предсказания одинаковы по точности')

Предсказание методом линейной регрессии точнее
